# Calculate Q-Rookley

- Input data "20172022_processed_1_3_4.csv", from 2017/7/1 to 2022/12/31, same as used in Figlewski

- Use 500 rolling window for P density (when calculating EPK)

- Use logreturn instead of log+1

- Use 
```
r.source('Q_Rookley_main_3.R')
```
and
```
'EPK_library_1.r'
```
to calculate Q_Rookley, along with 
```
get_btc_prices_2015_2022
```
for physical density.

- Interest rate: 0

- Moneyness range is [min moneyness, max moneyness]

- Bandwidth 0.3

- tau=35

In [1]:
import platform
plat = platform.system()
import locale
if plat == 'Darwin':
    print('Using Macos Locale')
    locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')
else:
    locale.setlocale(locale.LC_ALL, 'en_US.utf8')
import datetime
import pandas as pd
import sys
import matplotlib.pyplot as plt
import csv
import gc
import math
import numpy as np
import os
import pdb
import pickle
import statsmodels.api as sm
import time


import rpy2.robjects.packages as rpackages
from rpy2 import robjects

from scipy import stats
from scipy.integrate import simps
from scipy import interpolate

from src.helpers import save_dict, load_dict
from src.brc import BRC
from src.strategies import IronCondor
from src.quandl_data import get_btc_prices_2015_2022

from numpy.lib.function_base import insert

from statsmodels.nonparametric.kernel_regression import KernelReg as nadarajawatson

from sklearn.neighbors import KernelDensity

from pathlib import Path

def filter_sub(_df,  mindate, maxdate, tau):
    # Subset

    # Only calls, tau in [0, 0.25] and fix one day (bc looking at intra day here)
    # (_df['is_call'] == 1) & --> Dont need to filter for calls because that happens in the R script
    # Also need to consider put-call parity there
    sub = _df[(_df['date'] >= mindate) & (_df['date'] <= maxdate) & (_df['moneyness'] >= 0.5) & (_df['moneyness'] < 2) &(_df['mark_iv'] > 0)]# &

    if tau > 0:
        sub = sub[sub['tau'] == tau]

    nrows = sub.shape[0]
    if nrows == 0:
        raise(ValueError('Sub is empty'))
   
    sub['moneyness'] = round(sub['moneyness'], 3)
    sub['underlying_price'] = round(sub['underlying_price'], 2)

    sub = sub.drop_duplicates()
    
    print(sub.describe())

    if nrows > 50000:
        print('large df, subsetting')
        sub = sub.sample(10000)
        print(sub.describe())

    return sub

def prepare_confidence_band_data(df):

            # Old
                # Save for Confidence Band 
                    # Column 1 - Day
                    # Column 2 - Month
                    # Column 3 - Year
                    # Column 4 - Call Indicator
                    # Column 5 - Time Maturity in Days
                    # Column 6 - Strike
                    # Column 7 - IV
                    # Column 8 - Spot
                    # Column 9 - Risk Free Rate
                    # Column 10 - 
                    # Column 11 - Forward
                # 3	1	1997	0	49	3025	165.8	2861.706	0.031465	0.150026951	2863.26
                # Day Month Year Put Days Strike IV     Spot         Risk Free    Tau in Days (255)       Forward

    """
    # Mapping Table for Input

    # Column ... X
    # 1 - date
    # 2 - IV
    # 3 - 1 for Call, 0 for Put
    # 4 - Tau 
    # 5 - Strike
    # 6 - assigned meanwhile as put2call[,6]+ put2call[,7] -  put2call[,5]*exp(- mean(put2call[,8])* tau);
    # 7 - Spot
    # 8 - Probably Interest Rate
    # 9 - assigned meanwhile Moneyness: Spot/Strike
    """
    b = df[['date', 'mark_iv', 'putcall', 'tau', 'K', 'underlying_price', 'interest_rate', 'moneyness', 'instrument_price']]
    b['placeholder'] = 0
    b = b.reset_index(drop=True)
    # b['interest_rate'] = 0.00
    b['forward'] = 1

    # Reorder
    df_c = b[['date', 'mark_iv', 'putcall', 'tau', 'K', 'instrument_price', 'underlying_price', 'interest_rate', 'moneyness']]

    unique_maturities = df.days_to_maturity.unique()[0]
    fname = str('confidence_band_input_') + str(unique_maturities) + str('.csv')
    df_c.to_csv(fname, index = False)
    return fname

def bootstrap(conf_fname, phys_fname, rdate, tau, simmethod, r, out_dir, bw, lower, upper):
                            
    try:
        # MAKE SURE TO USE DIFFERENT 2ND INPUT FOR SP500!!
        moneyness, spd, epk, epk_lo, epk_up, logret, spd_logret, epk_logret, epk_logret_lo, epk_logret_up, epk_BS, tau, volas, spd_lo, spd_up, cdf_m, cdf_ret= r.bootstrap_epk(robjects.StrVector([conf_fname]), 
                                      robjects.StrVector([phys_fname]),
                                      robjects.StrVector(rdate),
                                      robjects.StrVector([out_dir]),
                                      robjects.FloatVector([bw]),
                                      robjects.FloatVector([lower]),
                                      robjects.FloatVector([upper])
                                     )
        # Can recover PD here as SPD/EPK
        spd_df = pd.DataFrame({'m': moneyness,
                               'spdy': spd,
                               'spdy_up': spd_up,
                               'spdy_dn': spd_lo,
                               'epk': epk,
                               'ret': logret,
                               'spd_ret': spd_logret,
                               'cb_ret_up': epk_logret_up,
                               'cb_ret_dn': epk_logret_lo,
                               'epk_ret': epk_logret,
                               'volatility': volas,
                               'epk_up': epk_up,
                               'epk_dn': epk_lo,
                               'cdf_m': cdf_m,
                               'cdf_ret': cdf_ret})

        return spd_df
    
    except Exception as e:
        print('Sim failed, proceeding with the next one... - ', simmethod)
        print('exception: ', e)
        
# Initiate R Objects
base = rpackages.importr("base") 
r = robjects.r
r.source('src/confidence_bands.R')
r.source('src/Q_Rookley_main_3.R')

# BTC prices
tdat = get_btc_prices_2015_2022()
tdat[::-1].to_csv('Data/BTC_USD_Quandl_2015_2022.csv', index = False)

bw_range = np.array([0.3])
r_bandwidth = bw_range[0]

simmethod = ['oldschool']

df = pd.read_csv('Data/20172022_processed_1_3_4.csv')
interest_rate_data = pd.read_csv('Data/IR_daily.csv')
interest_rate_data = interest_rate_data.rename(columns={'index': 'date','DTB3': 'interest_rate'})

df1=df[df['tau']==35]
dates=df1['date'].unique()
#df1=df[df['date']>=dates[9]]
#dates = df1['date'].unique()

dates1 = dates

error_i = []
for i0 in range(dates1.size):
    df2 = df1[df1['date']==dates1[i0]]
    
    curr_day = datetime.datetime.strptime(dates1[i0],'%Y-%m-%d')
    curr_day_starttime = curr_day.replace(hour = 0, minute = 0, second = 0, microsecond = 0)
    curr_day_endtime = curr_day.replace(hour = 23, minute = 59, second = 59, microsecond = 0)
    print(curr_day)

    df3 = pd.merge(df2, interest_rate_data, on='date', how='left')
    

    df3['interest_rate'] = 0
    df3['days_to_maturity'] = df3['tau']
    df3['tau'] = df3['tau']/365
    df3['underlying_price'] = df3['BTC_price']
    df3['instrument_price'] = df3['option_price']
    df3['putcall'][df3['putcall']=='C']=1
    df3['putcall'][df3['putcall']=='P']=0
    
    df_1 = df3[['date','BTC_price', 'K', 'interest_rate', 'tau', 'putcall', 'IV',  'moneyness',
                'days_to_maturity',  'underlying_price', 'instrument_price']]
    df_1['mark_iv']=df_1['IV']/100
    
    df_1.to_csv('Results/option_' + datetime.datetime.strftime(curr_day_starttime, '%Y%m%d') + '.csv')
    
    conf_fname = prepare_confidence_band_data(df_1)
    
    rdate = base.as_Date(curr_day.strftime('%Y-%m-%d'))
    rdate_f = base.format(rdate, '%Y-%m-%d')
    
    tau = df['tau'].unique()[0]
    
    spd_btc = bootstrap(conf_fname, 'Data/BTC_USD_Quandl_2015_2022.csv', rdate_f, tau, simmethod, r, 
                                     'Results/Rookley_Q_CB_1722_1_2_5_1_ttm35', r_bandwidth, df_1['moneyness'].min(), df_1['moneyness'].max())
    
    try :
        
        spd_btc.to_csv('Results/Rookley_Q_CB_1722_1_2_5_1_ttm35/btc_pk_'  + str(curr_day_starttime.__format__('%Y-%m-%d')) +'_bw_' + str(r_bandwidth) +  '.csv')
        
        print(spd_btc)
        
    except Exception as e:
        error_i.append(i0)

Using Macos Locale


/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
R[write to console]: 
Attaching package: ‘matlab’


R[write to console]: The following object is masked from ‘package:stats’:

    reshape


R[write to console]: The following objects are masked from ‘package:utils’:

    find, fix


R[write to console]: The following object is masked from ‘package:base’:

    sum


R[write to console]: KernSmooth 2.23 loaded
Copyright M. P. Wand 1997-2009

2017-08-25 00:00:00


/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_29145/3221028903.py:197: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='C']=1
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_29145/3221028903.py:198: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='P']=0
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_29145/3221028903.py:202: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

[1] "bandwidth: 0.3"
[1] "test no arbitrage condition"
[1] 44.694713  4.722982 66.258718
                [,1]      [,2]         [,3]        [,4]
sigmas -0.5824204022 0.9472216  0.070054019 -6.43554002
       -0.5770690419 0.9470256  0.060143871 -6.44798364
       -0.5717461662 0.9468297  0.049779231 -6.45616392
       -0.5664514737 0.9466311  0.038983358 -6.46012848
       -0.5611846673 0.9464270  0.027779355 -6.45992580
       -0.5559454549 0.9462150  0.016190155 -6.45560529
       -0.5507335488 0.9459926  0.004238524 -6.44721725
       -0.5455486659 0.9457575 -0.008052943 -6.43481295
       -0.5403905275 0.9455073 -0.020661833 -6.41844463
       -0.5352588589 0.9452399 -0.033565916 -6.39816554
       -0.5301533900 0.9449532 -0.046743149 -6.37402994
       -0.5250738545 0.9446454 -0.060171687 -6.34609310
       -0.5200199904 0.9443144 -0.073829888 -6.31441138
       -0.5149915394 0.9439586 -0.087696316 -6.27904216
       -0.5099882473 0.9435762 -0.101749756 -6.24004390
       -0.50500

...

[1] 44.694713  4.722982 66.258718
               [,1]      [,2]         [,3]         [,4]
sigmas -0.645833729 1.1556631 -1.655512625  3.064143067
       -0.633840804 1.1452783 -1.636430197  3.053413654
       -0.621990007 1.1350112 -1.617393663  3.042560165
       -0.610278008 1.1248619 -1.598405183  3.031588329
       -0.598701593 1.1148302 -1.579466826  3.020503728
       -0.587257659 1.1049159 -1.560580567  3.009311803
       -0.575943208 1.0951190 -1.541748290  2.998017847
       -0.564755344 1.0854394 -1.522971794  2.986627014
       -0.553691264 1.0758767 -1.504252788  2.975144319
       -0.542748259 1.0664308 -1.485592903  2.963574633
       -0.531923709 1.0571014 -1.466993685  2.951922693
       -0.521215076 1.0478884 -1.448456604  2.940193096
       -0.510619904 1.0387914 -1.429983052  2.928390305
       -0.500135814 1.0298101 -1.411574346  2.916518646
       -0.489760501 1.0209442 -1.393231732  2.904582313
       -0.479491730 1.0121935 -1.374956387  2.892585368
       -0.4693

        0.361166978 0.6744234  0.312567609  1.151745042
        0.365564842 0.6764273  0.319517217  1.133948479
        0.369943449 0.6784736  0.326338317  1.116023638
        0.374302968 0.6805614  0.333029722  1.097969824
        0.378643563 0.6826898  0.339590239  1.079786381
        0.382965399 0.6848579  0.346018676  1.061472693
        0.387268638 0.6870648  0.352313841  1.043028184
        0.391553437 0.6893096  0.358474538  1.024452316
        0.395819956 0.6915913  0.364499573  1.005744591
        0.400068349 0.6939089  0.370387752  0.986904547
        0.404298769 0.6962616  0.376137881  0.967931765
        0.408511368 0.6986483  0.381748767  0.948825859
        0.412706295 0.7010680  0.387219219  0.929586485
        0.416883699 0.7035198  0.392548048  0.910213335
        0.421043724 0.7060027  0.397734067  0.890706137
        0.425186515 0.7085157  0.402776092  0.871064658
        0.429312215 0.7110577  0.407672940  0.851288699
        0.433420963 0.7136278  0.412423435  0.83

/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_29145/3221028903.py:197: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='C']=1
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_29145/3221028903.py:198: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='P']=0
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_29145/3221028903.py:202: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

[1] 44.694713  4.722982 66.258718
               [,1]      [,2]         [,3]        [,4]
sigmas -0.847008738 1.2095968 -1.769607212  1.86543594
       -0.831343601 1.1980387 -1.760432417  1.89441841
       -0.815920080 1.1865234 -1.750961295  1.92290253
       -0.800730836 1.1750537 -1.741199133  1.95089175
       -0.785768857 1.1636320 -1.731151133  1.97838934
       -0.771027443 1.1522609 -1.720822424  2.00539838
       -0.756500184 1.1409429 -1.710218057  2.03192181
       -0.742180949 1.1296804 -1.699343012  2.05796239
       -0.728063863 1.1184757 -1.688202201  2.08352269
       -0.714143298 1.1073312 -1.676800470  2.10860513
       -0.700413858 1.0962491 -1.665142599  2.13321197
       -0.686870365 1.0852317 -1.653233310  2.15734530
       -0.673507850 1.0742812 -1.641077266  2.18100702
       -0.660321540 1.0633997 -1.628679076  2.20419891
       -0.647306848 1.0525893 -1.616043296  2.22692256
       -0.634459364 1.0418521 -1.603174433  2.24917941
       -0.621774847 1.0311900 -

        0.353229006 0.5985014  0.413780209  0.88690873
        0.357972008 0.6013471  0.418648403  0.84878591
        0.362692620 0.6042197  0.423226957  0.81048299
        0.367391053 0.6071171  0.427515328  0.77200635
        0.372067514 0.6100373  0.431513032  0.73336222
        0.376722207 0.6129783  0.435219641  0.69455673
        0.381355334 0.6159381  0.438634780  0.65559590
        0.385967094 0.6189145  0.441758126  0.61648563
        0.390557684 0.6219055  0.444589405  0.57723168
        0.395127296 0.6249092  0.447128391  0.53783973
        0.399676122 0.6279235  0.449374902  0.49831531
        0.404204350 0.6309464  0.451328799  0.45866382
        0.408712165 0.6339759  0.452989987  0.41889056
        0.413199751 0.6370099  0.454358407  0.37900070
        0.417667289 0.6400465  0.455434040  0.33899929
        0.422114956 0.6430837  0.456216899  0.29889125
        0.426542930 0.6461195  0.456707033  0.25868138
        0.430951382 0.6491518  0.456904524  0.21837436
        0.

In [2]:
print(error_i)

[3, 12, 14, 22]
